In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Experiments - Manage artifacts lineage.

* [Get started with Vertex AI Experiments](https://colab.sandbox.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb?hl=ko#scrollTo=title%3Ageneric%2Cgcp)

### Install Vertex AI SDK for Python and other required packages


In [1]:
%pip install --quiet --user google-cloud-aiplatform


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# @title Authentication to GCP
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [3]:
# @title Set GCP information
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [4]:
# @title Create a bucket.
BUCKET_URI = f"gs://mlops-0221"
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlops-0221/...
ServiceException: 409 A Cloud Storage bucket named 'mlops-0221' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [5]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [6]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://mlops-0221/
No changes made to gs://mlops-0221/


In [7]:
# @title Import libraries
import os
import uuid

import google.cloud.aiplatform as aiplatform

## Artifact lineage in experiment runs

In [8]:
# @title Create artifact lineage in experiment runs

EXPERIMENT_NAME = f"artifact-{uuid.uuid1()}"

aiplatform.init(project=PROJECT_ID, 
                location=LOCATION, 
                staging_bucket=BUCKET_URI,
                experiment=EXPERIMENT_NAME)

aiplatform.start_run("run-1")

Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/artifact-8df347cc-ef98-11ef-b0e9-42010a80000f-run-1 to Experiment: artifact-8df347cc-ef98-11ef-b0e9-42010a80000f


In [9]:
# @title Create a dataset and model artifacts

DATASET_URI = "gs://example/dataset.csv"
MODEL_URI = "gs://example/saved_model.pb"

# https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Artifact#google_cloud_aiplatform_Artifact_create

dataset_artifact = aiplatform.Artifact.create(
    schema_title="system.Dataset", display_name="example_dataset", uri=DATASET_URI
)

model_artifact = aiplatform.Artifact.create(
    schema_title="system.Model", display_name="example_modl", uri=MODEL_URI
)

In [11]:
# @title Create the artifact lineage

# https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_start_execution

with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    execution.assign_input_artifacts([dataset_artifact])

    aiplatform.log_params({"units": 256})
    aiplatform.log_metrics({"acc": 96.8})

    execution.assign_output_artifacts([model_artifact])

    aiplatform.log_metrics(
        {"lineage": execution.get_output_artifacts()[0].lineage_console_uri}
    )

In [ ]:
# @title Get the experiment results
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

In [13]:
# @title Visualize the artifact lineage

print("Open the following link:", execution.get_output_artifacts()[0].lineage_console_uri)

Open the following link: https://console.cloud.google.com/vertex-ai/locations/us-central1/metadata-stores/default/artifacts/2c69e933-16b2-46f6-a6cb-89af3bb40ec5?project=ai-hangsik


In [ ]:
# @title Delete the artifact lineage

try:
    dataset_artifact.delete()
except Exception as e:
    print(e)
try:
    model_artifact.delete()
except Exception as e:
    print(e)

In [ ]:
# @title Delete the experiment
exp = aiplatform.Experiment(EXPERIMENT_NAME)

try:
    exp.delete()
except Exception as e:
    print(e)

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
! rm -rf custom

delete_bucket = False

if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}